In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [19]:
standings_url = "https://www.baseball-reference.com/"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table#standings_AL')[0]


In [48]:
links = standings_table.find_all('a')
links = [l.get("href") for l in links] 
team_urls = [f"https://www.baseball-reference.com{l}" for l in links] 

team_url = team_urls[2] # first club page link
print(team_url)
data = requests.get(team_url) 

https://www.baseball-reference.com/teams/TOR/2022.shtml


In [34]:
data = requests.get(team_url)
soup = BeautifulSoup(data.text) # extract all information from page
links = soup.find_all('a') # find all links on page for the club
links = [l.get("href") for l in links] # get reference urls for those links
links = [l for l in links if l and (('schedule-scores' in l))]
links = [*set(links)][0]
sch_url = f"https://www.baseball-reference.com{links}"
print(sch_url)
data = requests.get(sch_url) 

https://www.baseball-reference.com/teams/NYY/2022-schedule-scores.shtml


In [47]:
sch = pd.read_html(data.text, match='Team Game-by-Game Schedule')[0]
sch = sch.drop(sch[sch['Gm#']=='Gm#'].index)
sch = sch.drop(sch[sch['Streak'].isna()].index)
sch = sch[['Gm#', 'Date', 'Tm', 'Unnamed: 4', 'Opp', 'W/L', 'R',
       'RA', 'Inn', 'W-L', 'Rank', 'GB', 'Win', 'Loss', 'Save', 'Time', 'D/N',
       'Attendance', 'Streak']]